# Importing Libraries

In [2]:
import pandas as pd
import matplotlib as plt

# Loading the Dataset

Data is downloaded to the `/data` subdirectory of the root project folder. We will use `pandas` to create a data frame with this data. 

In [3]:
data = pd.read_csv('data/AirPassengers.xls')